# 해시 테이블

## 1. Dictionary

* 매우 빠른 평균 삽입, 삭제, 탐색연산 제공
* key를 받으면 해시 함수로 인덱스 변환 후 value 반환
* 해시 함수에 의한 collision 발생 가능 -> collision resolution method

## 2. Hash Function

`-` Perfect Hash Function

* collision 없는 1:1 mapping function
* 이상적인 함수 : 상당히 어려움

`-` c-universal Hash Function

* 임의의 서로 다른 두 key값 x, y에 대해 $P(f(x) = f(y)) = \frac{c}{m}$인 경우
* 서로 다른 두 key값의 해시 함수 값이 같을 확률이 테이블의 크기에 반비례하는 해시함수

`-` 정수에 대한 hash function

**Division H.F**

* `k`는 `key`로 정수, `m`은 해시 테이블의 크기로 정수
* `f(k) = k%m` : 해시 테이블의 크기로 키 값을 나머지 연산
* `f(k) = (k%p)%m` : 일반적으로 적당히 큰 소수로 나머지연산 후 해시 테이블의 크기로 나머지 연산


**Multiplication H.F**

* `f(k) = ((ak) mod 2^w) >> (w-r)`

**Folding H.F**

* `key`값의 자리값들을 나눠서 연산

**Mid-Squeares H.F**

* `key`값을 적당히 연산한 후 결과의 중간 부분을 떼어내 함수로 리턴

**Extraction H.F**

* `key`값의 각 파트마다 임의의 digit을 떼어 붙여 연산

`-` 문자열에 대한 Hash Function

**Additive H.F**

* `key`의 각 문자열들의 단순 합으로 계산

**Rotating H.F**

* `<<, >>`(비트 쉬프트) 연산과 `^`(eXclusive OR) 연산을 반복

**Universal H.F**

* Rotating H.F에서 `h = ((h*a) + key[i])%p; return h%m`로 변환

`-` 좋은 해시 함수

1. fast compute
2. less collision

## 3. Collision Resolution method

### A. Open Addressing

충돌이 발생한 `key` 값을 다른 빈 슬롯을 찾아 그 슬록에 저장하는 방법


`-` Linear Probing 선형 탐색

* 충돌이 일어나면 밑에 있는 빈 슬롯 중 가장 근처에 있는 곳으로 저장(맨 아래일 경우 시작으로 복귀)
* Cluster : 연속적으로 key 값이 저장되어있을 경우 빈칸을 찾는 데에 시간이 오래 걸림 -> Cluster가 생기기 쉬움
* Search 연산 시 Cluster의 첫 번째 매핑으로 들어간 뒤 key가 일치하지 않으면 계속 내려감. 이 경우 수행 시간이 길어짐. 저장되어있지 않은 키값이 존재할 경우 빈 슬롯을 만남 -> None

> **연산**
>
> * `search(key)` : key값이 hash table에 있으면 value를 return.
> * `set(key, value = None)` : key값이 hash table에 있으면 value를 update -> key값이 hash table에 없으면 insert
> * `remove(key)` : key값이 있으면 제거

```{raw}
find_slot(key) :
    ## key 값이 있으면 slot 번호 리턴
    ## key 값이 없다면 key값이 삽입될 slot 번호 리턴
    i = f(key)
    start = i

    while (H[i] == occupied) and (H[i].key != key)
        i = (i+1)%m
        if i == start :
            return Full ## 꽉 차서 Full 자체를 반환

    return i

set(key, value = None) :
    i = find_slot(key)
    
    if i == Full : ## H가 꽉 차있으면 H의 사이즈를 키워야 함
        return None ## Hash Table이 꽉 차서 연산 수행 불가

    if H[i] is occupuied :
        H[i].value = value
    else :
        H[i].key, H[i].value = key, value

    return key ## 성공적으로 set 연산 수행

search(key) :
    i = find_slot(key)

    if i == Full :
        return None ## 꽉차서 없음

    if H[i] is occupied :
        return H[i].value
    else :
        return None ## H 안에 키 없음
```

* `remove`의 경우에는 조금 더 복잡합 : 해시 함수 규칙이 깨져서 찾지 못하게 만들 수 있음 -> 탐색의 고리가 끊어질 수 있음

|key|index|
|:-:|:-:|
|B9|0|
||1|
|A2|2|
|A3|3|
|B2|4|
|A5|5|
|B5|6|
|C2|7|
||8|
|A9|9|

&nbsp;

> A3를 지우면, Linear Probing에서 B2를 찾기 전에 공란이 있기 때문에 없다고 반환하게 됨
>
> B2를 위로 올리고, C2도 위로 올려야 함 -> 충돌이 발생되는 원래 공간에 해당되는 애들을 다 옮겨야 함
>
> 경우에 따라선 한 바퀴를 돌아서 위에 옮겨야 하는 애들이 생길 수도 있음 -> 원래 해시값 k, 빈칸 i, 옮길 값 j -> k < i < j 또는 j < k < i

```{raw}
remove(key) :
    i = find_slot(key)

    if H[i] is not occupied :
        return None ## 키 값이 없음

    j = i ## H[i] : empty slot, H[j] : 옮겨줘야 할 slot

    while True :
        H[i] = None ## empty slot

        while True :
            j = (j+1)%m ## j를 증가시켜가며 H[i]로 이사할 H[j]를 찾음
            if H[j] is not occupied :
                return key ## 비어있는 셀을 발견했으면 함수 종료, 키 반환
            
            k = f(H[j].key) ## 원래 key의 해시값

            if (k < i < j) or (j < k < i) or (i < j < k) :
                break ##
        
        H[i] = H[j] ## 이동
        i = j ## 빈 슬롯 위치 바뀜
```

> 클러스터의 길이가 길면 길수록, 연산에 소요되는 시간이 길어짐 (find_slot 오래 걸림)
>
> 클러스터 길이를 결정하는 것은 해시 함수 / collision resolution method에 달려 있음
>
> Linear Probing은 collision이 발생할 경우 Cluster의 길이가 무조건 하나 증가하게 됨 -> 좋은 collision resolution method가 아님.
>
> **군집에 다른 키들이 삽입될 때 긴 탐색이 강제됨**

`-` Quadratic Probing

* Collision 발생 시 하나씩 내리지 않고 제곱항의 수만큼 내림
* 연속된 인덱스를 할당시키지 않는다는 점에서 클러스터의 길이는 대체로 줄어드나, Secondary Clustering은 그대로 남아 있음

`-` Double Hashing

* 두 개의 해시 함수를 사용하여 해시 값을 생성
* 하나의 해시 함수를 사용했을 때 충돌 `f(key)` -> `f(key) + g(key)` -> `f(key) + 2*g(key)` -> ...

`-` Cluster Size

* Hash Function
* Collision Resolution Method
* Load Factor : $\frac{n}{m}$ -> 평균적으로 50% 이상 맞추도록 하면 클러스터의 평균 사이즈가 $O(1)$ -> 연산도 $O(1)$
> `m : number of slot`, `n : number of item` -> 수행 시간은 Load Factor에 비례함. 많이 차있을 수록 충돌이 많이 일어남 ㅇㅇ

* $\frac{\text{collisions}}{n} = $ 충돌 비율로 `set, remove, search` 연산의 효율성을 판단할 수 있음

### B. Chaining

* Collision이 생기면 그냥 하나의 슬롯에 여러 개의 아이템을 저장하면 되잖아?
* 각 슬롯에 한 방향 연결 리스트를 생성
* 군집은 생기지 않음, 연결 리스트가 길어지면 비효율적 -> c-universal hash function이면 연결 리스트의 길이는 평균적으로 $O(1)$임이 알려져 있음.

> * `set(key)` : head node를 추가 -> $O(1)$
> * `search(key)`, `remove(key)` : 연결 리스트의 길이 == 충돌 key의 평균 갯수에 의존 -> $O(1)$

## 정리

* 좋은 Hash Function : c-universal H.F
* 충분한 빈 슬롯 유지 : collision rate

> Open addressing / Chaining 모두 상수 시간 안에 주요 연산이 가능함